In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sqlite3
from dataclasses import fields, asdict
from dataclass_csv import DataclassReader

from app.types.classes import *

In [ ]:
from typing import Type
def csv_to_list(filename:str, cls: Type, hdrs:str) -> list:
    with open(f"csvdata/{filename}.csv") as f:
        reader = DataclassReader(f, cls)
        for hdr in hdrs.split("|"):
            reader.map(hdr).to(hdr.lower())
        return list(reader)

In [ ]:
players = csv_to_list("players", Player, "Code|Surname|Initial|Active|FirstName")
print(len(players), "players loaded")

In [ ]:
seasons = csv_to_list("seasons", Season, "Year|Played|Won|Lost|Drawn|Tied|NoResult|MaxPossibleGames")
print(len(seasons), "seasons loaded")

In [ ]:
def insert_sql(cls: type, tablename: str) -> str:
    cols = ", ".join(field.name for field in fields(cls))
    vals = ", ".join(f":{field.name}" for field in fields(cls))
    return f"INSERT INTO {tablename} ({cols}) VALUES ({vals})"

In [ ]:
def count(tablename: str) -> int:
    csr = conn.execute(f"SELECT COUNT(*) FROM {tablename}")
    return csr.fetchone()[0]

In [ ]:
conn = sqlite3.connect(":memory:")

In [ ]:
with open("db_schema.sql") as f:
    conn.executescript(f.read())

In [ ]:
print(count("players"))


In [ ]:
sql = insert_sql(Player, "players")

In [ ]:
conn.executemany(sql, [asdict(player) for player in players])

In [ ]:
print(count("players"))

In [ ]:
conn.execute("select * from players where active limit 10").fetchall()